In [74]:
import pandas as pd
import numpy as np
import re

from sklearn.svm import SVC  
from sklearn.preprocessing import MinMaxScaler

pd.set_option('display.max_columns', None)

In [75]:
train = pd.read_csv('./Data/train_cleaned_v0.0.csv')
test = pd.read_csv('./Data/test_cleaned_v0.0.csv')
train_raw = pd.read_csv('./Data/training_set_values.csv')
test_raw = pd.read_csv('./Data/test_set_values.csv')

labels = pd.read_csv('./Data/training_set_labels.csv')
train = train.merge(labels, on="id")
target = train.pop("status_group")

train['train'] = 1
test['train'] = 0

data = pd.concat([train,test])

data['date_recorded'] = pd.to_datetime(data['date_recorded'])
data['date_recorded'] = data['date_recorded'].dt.year

# Execute following cell to impute missing values with median

In [73]:
num_features = ['date_recorded', 'latitude','longitude','amount_tsh', 'gps_height', 'population', 'operation_years', 'construction_year']

data = data.drop(['gps_height_imp_random_choice', 'gps_height_imp_normal', 'population_imp_normal',
               'population_imp_random_choice', 'construction_year_imp_normal', 'construction_year_imp_random_choice',
               'amount_tsh_imp_normal', 'amount_tsh_imp_random_choice', 'longitude_imp_normal',
                 'longitude_imp_random_choice', 'latitude_imp_normal', 'latitude_imp_random_choice',
                 'operation_years_imp_normal', 'operation_years_imp_random_choice'], axis = 1)

# Execute following cell to impute missing values with random sample from normal distribution

In [54]:
num_features = ['date_recorded','latitude_imp_normal','longitude_imp_normal','amount_tsh_imp_normal',
                'gps_height_imp_normal', 'population_imp_normal', 'operation_years_imp_normal',
               'construction_year_imp_normal']

data = data.drop(['gps_height_imp_random_choice', 'gps_height', 'population',
               'population_imp_random_choice', 'construction_year', 'construction_year_imp_random_choice',
               'amount_tsh', 'amount_tsh_imp_random_choice', 'longitude',
                 'longitude_imp_random_choice', 'latitude', 'latitude_imp_random_choice',
                 'operation_years', 'operation_years_imp_random_choice'], axis = 1)

# Execute following cell to impute missing values with random choice from empirical distribution

In [76]:
num_features = ['date_recorded','latitude_imp_random_choice','longitude_imp_random_choice','amount_tsh_imp_random_choice',
                'gps_height_imp_random_choice', 'population_imp_random_choice',
                'operation_years_imp_random_choice', 'construction_year_imp_random_choice']

data = data.drop(['gps_height', 'gps_height_imp_normal', 'population_imp_normal',
               'population', 'construction_year_imp_normal', 'construction_year',
               'amount_tsh_imp_normal', 'amount_tsh', 'longitude_imp_normal',
                 'longitude', 'latitude_imp_normal', 'latitude',
                 'operation_years_imp_normal', 'operation_years'], axis = 1)

In [78]:
# Scale numerical features
scaler = MinMaxScaler()
scaler.fit(data[num_features])
data[num_features] = scaler.transform(data[num_features])

In [79]:
# Get categorical features for OneHotEncoding
cat_features = list(data.columns)
for feature in num_features:
    cat_features.remove(feature)

In [80]:
# Remove train and id column for OneHotEncoding
cat_features.remove('train')
cat_features.remove('id')

In [81]:
# Reduce dimension of some categorical features
dim_red_features = 'funder installer scheme_name lga ward subvillage wpt_name'.split()
for feature in dim_red_features:
    train = train.assign(count = train.groupby(feature)[feature].transform('count')).sort_values(by = ['count',feature], ascending = [False,True])
    top_values = train.drop_duplicates('count')
    top_values = list(top_values.nlargest(10, 'count')[feature])
    data[feature] = data[feature].apply(lambda x: x if (x in top_values) | (str(x) == 'nan') else 'Others')

In [82]:
# Convert factorized categorical features to strings
data[cat_features] = data[cat_features].applymap(str)

In [83]:
# OneHotEncoding of categorical features
data = pd.concat([data, pd.get_dummies(data[cat_features], dummy_na=True)], axis=1)
data = data.drop(cat_features, axis = 1)

In [84]:
# extract training/test sets
train_df = data[data["train"] == 1]
test_df = data[data["train"] == 0]
train_df.drop(["train"], axis=1, inplace=True)
train_df.drop(['id'],axis=1, inplace=True)
test_df.drop(["train"], axis=1, inplace=True)

id_test = test_df['id']
test_df.drop(['id'],axis=1, inplace=True)

C:\Anaconda\envs\py36\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# Train model and predict values

In [85]:
svm = SVC()
svm.fit(train_df, target)

C:\Anaconda\envs\py36\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [86]:
predictions = svm.predict(test_df)

predictions = pd.DataFrame(predictions)
predictions['id'] = id_test
predictions.columns = ['status_group','id']
predictions = predictions[['id','status_group']]
# convert into submission format

formatsub = pd.read_csv('./Data/submission_format.csv')
submission_format = pd.merge(formatsub, predictions, on=['id'], how='inner')
submission_format.drop(['status_group_x'],axis=1,inplace=True)
submission_format.columns = ['id','status_group']

submission_format.to_csv('./Results/submission_format_SVM_empirical.csv', index=False)
submission_format.head()

,id,status_group
0,50785,non functional
1,51630,functional
2,17168,non functional
3,45559,non functional
4,49871,functional


In [70]:
train_df.iloc[0][150:250]

ward_11                      0.0
ward_136                     0.0
ward_16                      0.0
ward_17                      0.0
ward_28                      0.0
ward_30                      0.0
ward_43                      0.0
ward_615                     0.0
ward_72                      0.0
ward_Others                  1.0
ward_nan                     0.0
public_meeting_0             1.0
public_meeting_1             0.0
public_meeting_2             0.0
public_meeting_nan           0.0
scheme_management_0          1.0
scheme_management_1          0.0
scheme_management_10         0.0
scheme_management_11         0.0
scheme_management_12         0.0
scheme_management_2          0.0
scheme_management_3          0.0
scheme_management_4          0.0
scheme_management_5          0.0
scheme_management_6          0.0
scheme_management_7          0.0
scheme_management_8          0.0
scheme_management_9          0.0
scheme_management_nan        0.0
scheme_name_1                0.0
          